In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import random

pi = 3.14159265359

maxval=1e9
minval=1e-9

import OptimizedDataGenerator as DG
from loss import *
from models import *
import utils

2024-08-28 20:30:53.601443: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 20:30:53.601523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 20:30:53.603280: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 20:30:53.612514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-28 20:30:57.165034: W tensorflow/compiler/tf2

In [2]:
batch_size = 5000
val_batch_size = 5000
train_file_size = 104
val_file_size = 6

## Make the tf records

The following happened when training with dataset18-3D

When using the OptimizedDataGenerator to create tfrecord for training, the tfrecord would be "broken" after the first use (training with it will only lead to a plateau of loss at around 10k). Recreateing the tfrecord and using it will solve this issue, where the model converges as expected. This bug can be avoided by using optimized DG to create tfrecord before each new training (instead of using used tfrecord). I discussed with Arghya on this.

In [3]:
data_directory_path = "/depot/cms/users/kuang14/Smart_Pixel/dataset18-3D/recon3D/"
labels_directory_path = "/depot/cms/users/kuang14/Smart_Pixel/dataset18-3D/labels/"

Create a new tfrecord can avoid the problem of not converging. 

In [4]:
tfrecords_dir_train = "/depot/cms/users/kuang14/Smart_Pixel/tf_records/tfrecords_2t_dataset18_bs5000_train"
tfrecords_dir_validation = "/depot/cms/users/kuang14/Smart_Pixel/tf_records/tfrecords_2t_dataset18_bs5000_val"

In [5]:
utils.safe_remove_directory(tfrecords_dir_train)
utils.safe_remove_directory(tfrecords_dir_validation)

Directory /depot/cms/users/kuang14/Smart_Pixel/tf_records/tfrecords_2t_dataset18_bs5000_train3 does not exist and cannot be removed.
Directory /depot/cms/users/kuang14/Smart_Pixel/tf_records/tfrecords_2t_dataset18_bs5000_val3 does not exist and cannot be removed.


In [6]:
validation_generator = DG.OptimizedDataGenerator(
    data_directory_path = data_directory_path,
    labels_directory_path = labels_directory_path,
    is_directory_recursive = False,
    file_type = "parquet",
    data_format = "3D",
    batch_size = val_batch_size,
    file_count = val_file_size,
    to_standardize= True,
    include_y_local= False, 
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    input_shape = (2,13,21), # (20,13,21),
    transpose = (0,2,3,1),
    shuffle = False, 
    files_from_end=True,

    tfrecords_dir = tfrecords_dir_validation,
    use_time_stamps = [0, 19], #-1
    max_workers = 2 # Don't make this too large (will use up all RAM)
)

Directory /depot/cms/users/kuang14/Smart_Pixel/tf_records/tfrecords_2t_dataset18_bs5000_val3 does not exist and cannot be removed.


Saving batches as TFRecords: 100%|██████████| 59/59 [00:07<00:00,  7.50it/s]


In [7]:
train_generator = DG.OptimizedDataGenerator(
    data_directory_path = data_directory_path,
    labels_directory_path = labels_directory_path,
    is_directory_recursive = False,
    file_type = "parquet",
    data_format = "3D",
    batch_size = batch_size,
    file_count = train_file_size,
    to_standardize= True,
    include_y_local= False, 
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    input_shape = (2,13,21), # (20,13,21),
    transpose = (0,2,3,1),
    shuffle = False, 
    files_from_end=True,

    tfrecords_dir = tfrecords_dir_validation,
    use_time_stamps = [0, 19], #-1
    max_workers = 2 # Don't make this too large (will use up all RAM)
)

Directory /depot/cms/users/kuang14/Smart_Pixel/tf_records/tfrecords_2t_dataset18_bs5000_val3 is removed...


Saving batches as TFRecords: 100%|██████████| 1028/1028 [02:13<00:00,  7.71it/s]


In [8]:
1

1